# Decompositions and least squares


In this lecture, we look at several factorizations of a matrix. For a square or rectangular matrix $A ∈ ℝ^{m × n}$ with $m ≥ n$, we consider:
1. The _QR decomposition_
$$
A = \underbrace{Q}_{m × m} \underbrace{R}_{m × n} = \begin{bmatrix} 𝐪_1 | \cdots | 𝐪_m \end{bmatrix} \begin{bmatrix} × & \cdots & × \\ & \ddots & \vdots \\ && × \\ &&0 \\ &&\vdots \\ && 0 \end{bmatrix} 
$$
where $Q$ is orthogonal ($Q^⊤Q = I$, $𝐪_j ∈ ℝ^m$) and $R$ is _right triangular_.

2. The _reduced QR decomposition_
$$
A = \underbrace{Q̂}_{m × n} \underbrace{R̂}_{m × m} = \begin{bmatrix} 𝐪_1 | \cdots | 𝐪_n \end{bmatrix} \begin{bmatrix} × & \cdots & × \\ & \ddots & \vdots \\ && ×  \end{bmatrix} 
$$
where $Q$ has orthogonal columns ($Q^⊤Q = I$, $𝐪_j ∈ ℝ^m$) 

For a square matrix we consider the _PLU decomposition_:
$$
A = P^⊤ LU
$$
where $P$ is a permutation matrix, $L$ is lower triangular and $U$ is upper triangular.

Finally, for a square, _symmetric positive definite_ ($𝐱^⊤ A 𝐱 > 0$ for all $𝐱 ∈ ℝ^n$) 
matrix we consider the _Cholesky decomposition_:
$$
A = L L^⊤
$$

The importance of these decomposition for square matrices is that their component pieces are easy to invert on a computer:
$$
\begin{align*}
A = P^⊤ LU &\Rightarrow\qquad A^{-1}𝐛 = U^{-1} L^{-1} P 𝐛 \\
A = QR &\Rightarrow\qquad A^{-1}𝐛 = R^{-1} Q^\top 𝐛 \\
A = L L^⊤ &\Rightarrow\qquad A^{-1}𝐛 = L^{-⊤} L^{-1} 𝐛
\end{align*}
$$
and we saw last lecture that triangular and orthogonal matrices are easy to invert when applied to a vector $𝐛$.
For rectangular matrices we will see that they lead to efficient solutions to the _least squares problem_: find
$𝐱$ that minimizes the 2-norm
$$
\| A 𝐱 - 𝐛 \|.
$$

In this lecture we discuss the followng:

1. QR,  Reduced QR, and least squares: We discuss the QR decomposition and its usage in solving least squares problems.
We discuss computation of the Reduced QR decomposition using Gram–Schmidt, and the Full QR decomposition using 
Householder reflections.
2. PLU decomposition: we discuss how the LU decomposition can be computed using Gaussian elimination, and the computation of
the PLU decomposition via Gaussian elimination with pivoting.
3. Cholesky decomposition: we introduce symmetric positive definite matrices and show that their LU decomposition can be re-interpreted
as a Cholesky decomposition.

In [1]:
using LinearAlgebra, Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423


## 1. QR, Reduced QR, and least squares

Here we consider rectangular matrices with more rows than columns.
A QR decomposition decomposes a matrix into an orthogonal matrix $Q$ times a right triangular matrix $R$. 
Note the QR decomposition contains within it the reduced QR decomposition:
$$
A = QR = \begin{bmatrix} Q̂ | 𝐪_{n+1} | ⋯ | 𝐪_m \end{bmatrix} \begin{bmatrix} R̂ \\  𝟎_{m-n × n} \end{bmatrix} = Q̂ R̂.
$$

### Relationship with least squares

We can use it to solve a least squares problem using the norm-preserving property (see PS3) of orthogonal matrices:
$$
\| A 𝐱 - 𝐛 \| = \| Q R 𝐱 - 𝐛 \| = \| R 𝐱 - Q^⊤ 𝐛 \| = \left \| 
\begin{bmatrix} R̂ \\ 𝟎_{m-n × n} \end{bmatrix} 𝐱 - \begin{bmatrix} Q̂^⊤ \\ 𝐪_{n+1}^⊤ \\ \vdots \\ 𝐪_m^⊤ \end{bmatrix}     𝐛 \right \|
$$
Now note that the rows $k > n$ are independent of $𝐱$ and are a fixed contribution. Thus to minimise this norm it suffices to
drop them and minimise:
$$
\| R̂ 𝐱 - Q̂^⊤ 𝐛 \|
$$
This norm is minimisable if it is attained. Provided the column rank of $A$ is full, $R̂$ will be invertible (Exercise: why is this?).
Thus we have the solution
$$
𝐱 = R̂^{-1} Q̂^⊤ 𝐛
$$

Let's look at an example. Suppose we want to find noisy data by a quadratic
$$
p(x) = a + bx + cx^2
$$
That is, we want to choose $a,b,c$ at data samples $x_1, \ldots, x_m$ so that the following is true:
$$
a + b x_k + c x_k^2 ≈ f_k
$$
where $f_k$ are given by data. We can reinterpret this as a least squares problem: minimise the norm
$$
\left\| \begin{bmatrix} 1 & x_1 & x_1^2 \\ \vdots & \vdots & \vdots \\ 1 & x_m & x_m^2 \end{bmatrix}
\begin{bmatrix} a \\ b \\ c \end{bmatrix} - \begin{bmatrix} f_1 \\ \vdots \\ f_m \end{bmatrix} \right \|
$$
We can solve this using the QR decomposition:

In [2]:
x = range(0,1; length=100) # 1000 points
f = 2 .+ x .+ 2x.^2 .+ 0.1 .* randn.() # Noisy quadratic

A = [ones(length(x)) x x.^2]
Q,R̂ = qr(A)
Q̂ = Q[:,1:3] # Q represents full orthogonal matrix so we take first 3 columns

a,b,c = R \ Q̂'f

LoadError: UndefVarError: R not defined

We can visualise the fit:

In [3]:
p = x -> a + b*x + c*x^2

scatter(x, f; label="samples", legend=:bottomright)
plot!(x, p.(x); label="quadratic")

LoadError: UndefVarError: b not defined

Note that `\` with a rectangular system does least squares by default:

In [4]:
A \ f

3-element Vector{Float64}:
 2.0061494545254637
 1.0329624031651745
 1.9524446508610185

### Gram–Schmidt and reduced QR

We will now reinterpret 

### Householder reflections and QR





## 2. PLU Decomposition


## 3.